INORDER TO GET THE WEIGHTS OF THE MODELS DOWNLOAD THE WEIGHTS FOLDERS FROM THIS DRIVE LINK: https://drive.google.com/drive/folders/1WpRbsJGyqBrFrb1nWB6HCSIRjkUWWtvT?usp=share_link
<BR> AND PLACE THEM IN THE **'WEIGHTS'** FOLDER. PASTE THE WEIGHTS ALONG WITH THE FOLDERS OF THE PRE-TRAINED MODEL WEIGHTS IN THE **'WEIGHTS'** FOLDER.

FOR THE FID SCORE COMPILATION DOWNLOAD THE KAGGLE DATASET FROM THIS LINK:- https://www.kaggle.com/datasets/splcher/animefacedataset<BR> AND PASTE THE UNZIPPED FOLDER'S IMAGES IN THIS PATH: **'Inception-Score\data\real_images_64'**

# CREATE ENV FOR THE PROJECT

In [ ]:
! conda create --name torch_prog --file requirements.txt

# IMPORT LIBERARIES

In [1]:
from start import init_train
from utils.weights import LoadWeights
from utils.images import SaveImages
from math import ceil
import random
import PIL

d:\miniconda\envs\torch_prog\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# UTILITIES

In [2]:
def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols)))
   return canvas

# RESIZING THE INPUT IMAGES TO 64X64

In [ ]:
import os
from PIL import Image
path = 'E:/pythonProject/BTP/Inception-Score/data/real_images'
files = os.listdir(path)
for file in files:
    # Open the image file
    image = Image.open(path+"/"+file)

    # Resize the image to a new size
    new_size = (64, 64)
    resized_image = image.resize(new_size)

    # Save the resized image to a file
    resized_image.save('E:/pythonProject/BTP/Inception-Score/data/fake_images2/'+file)


# GENERATE IMAGES STYLEGAN

In [4]:
# Loading models and the latest weights without loading the dataset
Trainer = init_train("configs/StyleGAN_64.json", load_dataset=False)

# Loading custom weights with an inaccurate match
LoadWeights(Trainer, 'weight/StyleGAN StyleGAN 64/weight 420.pth')

# Save 10 randomly generated images to the img folder
SaveImages(Trainer, dir='Inception-Score/data/fake_images_64', cnt=10)

Generator(
  (mapping): Mapping(
    (blocks): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (2): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (3): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (4): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (5): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (6): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1):

100%|██████████| 10/10 [00:05<00:00,  1.79it/s]


In [ ]:
path = 'Inception-Score/data/fake_images_64/'
imgs = os.listdir(path)

In [ ]:
images = [PIL.Image.open(path+f'{imgs[i]}') for i in random.sample(range(50000), 42)]
createImageGrid(images, scale=1, rows=6)

# GENERATE IMAGES STYLEGAN2

In [ ]:
# Loading models and the latest weights without loading the dataset
Trainer = init_train("configs/StyleGAN2.json", load_dataset=False)

# Loading custom weights with an inaccurate match
LoadWeights(Trainer, 'weight/StyleGAN StyleGAN2 64/weight 79.pth')

# Save 10 randomly generated images to the img folder
SaveImages(Trainer, dir='Inception-Score/data/fake_images2_64', cnt=50000)

In [ ]:
images = [PIL.Image.open(path+f'{imgs[i]}') for i in random.sample(range(50000), 42)]
createImageGrid(images, scale=1, rows=6)

In [27]:
path = 'Inception-Score/data/fake_images2_64/'
imgs = os.listdir(path)

# REAL IMAGES

In [ ]:
path = 'Inception-Score/data/real_images_64/'
imgs = os.listdir(path)

In [ ]:
images = [PIL.Image.open(path+f'{imgs[i]}') for i in random.sample(range(50000), 42)]
createImageGrid(images, scale=1, rows=6)

# FID SCORE

## FID SCORE FOR STYLEGAN MODEL

In [ ]:
! python -m pytorch_fid Inception-Score/data/fake_images_64 Inception-Score/data/real_images_64 --device cuda:0 --dims 64

## FID SCORE FOR STYLEGAN2 MODEL

In [ ]:
! python -m pytorch_fid Inception-Score/data/fake_images2_64 Inception-Score/data/real_images_64 --device cuda:0 --dims 64

# SSIM SCORE

In [ ]:
import os
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from torchmetrics import StructuralSimilarityIndexMeasure
real_dir = "E:/pythonProject/BTP/Inception-Score/data/real_images_64/"
fake_dir = "E:/pythonProject/BTP/Inception-Score/data/fake_images2_64/"

# Define the transforms to be applied to the images
transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Load the real images
real_imag = os.listdir(real_dir)
fake_img = os.listdir(fake_dir)
real_images = []
for i in tqdm(range(50000)):
    image_path = real_dir + f"{real_imag[i]}"
    image = Image.open(image_path)
    image = transforms(image)
    real_images.append(image)

# Load the fake images
fake_images = []
for i in tqdm(range(50000)):
    image_path = fake_dir + f"{fake_img[i]}"
    image = Image.open(image_path)
    image = transforms(image)
    fake_images.append(image)

ssim = StructuralSimilarityIndexMeasure(data_range=1.0)

In [ ]:
# Calculate the SSIM score
total_ssim = 0
for i in tqdm(range(50000)):
    real_image = real_images[i]
    fake_image = fake_images[i]
    ssim_score = ssim(fake_image.unsqueeze(0), real_image.unsqueeze(0))
    total_ssim += ssim_score.item()

avg_ssim = total_ssim / 50000
print("Average SSIM score:", avg_ssim)